In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Swing States

- Pennsylvania
- Michigan
- Wisconsin
- Georgia
- Arizona
- North Carolina
- Nevada

In [2]:
# Load the most recent 538 Data

df = pd.read_csv("https://projects.fivethirtyeight.com/polls/data/president_polls.csv")
pd.set_option("display.max_columns", None)
df.head()



,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,fte_grade,methodology,transparency_score,state,start_date,end_date,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,endorsed_candidate_id,endorsed_candidate_name,endorsed_candidate_party,question_id,sample_size,population,subpopulation,population_full,tracking,created_at,notes,url,source,internal,partisan,race_id,cycle,office_type,seat_number,seat_name,election_date,stage,nationwide_batch,ranked_choice_reallocated,ranked_choice_round,party,answer,candidate_id,candidate_name,pct
0,85921,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B-,Online Panel,3.0,NaN,2/9/24,2/11/24,NaN,NaN,NaN,NaN,NaN,NaN,192016,6164.0,rv,NaN,rv,True,2/12/24 15:53,NaN,https://pro.morningconsult.com/trackers/2024-p...,NaN,NaN,NaN,8914,2024,U.S. President,0,NaN,11/5/24,general,False,False,NaN,DEM,Biden,19368,Joe Biden,42.0
1,85921,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B-,Online Panel,3.0,NaN,2/9/24,2/11/24,NaN,NaN,NaN,NaN,NaN,NaN,192016,6164.0,rv,NaN,rv,True,2/12/24 15:53,NaN,https://pro.morningconsult.com/trackers/2024-p...,NaN,NaN,NaN,8914,2024,U.S. President,0,NaN,11/5/24,general,False,False,NaN,REP,Trump,16651,Donald Trump,43.0
2,85922,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B-,Online Panel,3.0,NaN,2/8/24,2/10/24,NaN,NaN,NaN,NaN,NaN,NaN,192017,4000.0,rv,NaN,rv,True,2/12/24 15:53,NaN,https://pro.morningconsult.com/trackers/2024-p...,NaN,NaN,NaN,8914,2024,U.S. President,0,NaN,11/5/24,general,False,False,NaN,DEM,Biden,19368,Joe Biden,42.0
3,85922,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B-,Online Panel,3.0,NaN,2/8/24,2/10/24,NaN,NaN,NaN,NaN,NaN,NaN,192017,4000.0,rv,NaN,rv,True,2/12/24 15:53,NaN,https://pro.morningconsult.com/trackers/2024-p...,NaN,NaN,NaN,8914,2024,U.S. President,0,NaN,11/5/24,general,False,False,NaN,REP,Trump,16651,Donald Trump,44.0
4,85923,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B-,Online Panel,3.0,NaN,2/7/24,2/9/24,NaN,NaN,NaN,NaN,NaN,NaN,192018,4000.0,rv,NaN,rv,True,2/12/24 15:53,NaN,https://pro.morningconsult.com/trackers/2024-p...,NaN,NaN,NaN,8914,2024,U.S. President,0,NaN,11/5/24,general,False,False,NaN,DEM,Biden,19368,Joe Biden,41.0


In [3]:
#Define a function to grab the data by state


def get_state_data(state,df):
    state_data = df[df['state'] == state]
    return state_data.copy()

pa = get_state_data('Pennsylvania',df)
mi = get_state_data('Michigan',df)
wi = get_state_data('Wisconsin',df)
ga = get_state_data('Georgia',df)
az = get_state_data('Arizona',df)
nc = get_state_data('North Carolina',df)
nv = get_state_data('Nevada',df)
states = [pa,mi,wi,ga,az,nc,nv]

In [4]:
def get_candidate_data(candidate,df):
    candidate_data = df[df['candidate_name'] == candidate]
    return candidate_data.copy()

biden_states = [get_candidate_data('Joe Biden',state) for state in states]
trump_states = [get_candidate_data('Donald Trump',state) for state in states]
kennedy_states = [get_candidate_data('Robert F. Kennedy',state) for state in states]

biden_pa = biden_states[0]
trump_pa = trump_states[0]
kennedy_pa = kennedy_states[0]

def time_series(df, name, state, x='start_date', y='pct'):
    df = df.copy()
    df[x] = pd.to_datetime(df[x],format='%m/%d/%y').to_numpy()
    fig = px.scatter(df, x=x, y=y, title=f'{name} Polling in {state}')
    #Add least squares line
    m, b = np.polyfit(df.index, df[y], 1)
    fig.add_scatter(x=df[x], y=m*df.index+b, mode='lines', name='Least Squares')
    #Add traceline 
    fig.update_traces(hoverinfo='text+name', mode='lines+markers',)
    fig.update_layout(hovermode="x unified")
    #Add horizontal line at 50%
    fig.add_hline(y=50, line_dash="dot", line_color="red", annotation_text="50%")
    return fig.show()
#Make time series plot for Biden in PA
time_series(biden_pa, 'Joe Biden', 'Pennsylvania')
time_series(trump_pa, 'Donald Trump', 'Pennsylvania')
time_series(kennedy_pa, 'Robert F. Kennedy', 'Pennsylvania')


In [5]:
#Make a biden time series plot for all states
for i in range(len(states)):
    time_series(biden_states[i], 'Joe Biden', states[i]['state'].iloc[0])

# #Make a trump time series plot for all states
# for i in range(len(states)):
#     time_series(trump_states[i], 'Donald Trump', states[i]['state'].iloc[0])

# #Make a kennedy time series plot for all states
# for i in range(len(states)):
#     time_series(kennedy_states[i], 'Robert F. Kennedy', states[i]['state'].iloc[0])

In [6]:
#Zoom in on Michigan for Biden
time_series(biden_states[1], 'Joe Biden', states[1]['state'].iloc[0])

## How has the war in Gaza impacted Joe Biden's polling for Michigan?

- Michigan has a large Muslim population, which makes a significant proportion of Joe Biden's base.

## Did RFK Jr's Superbowl Ad Change anything?

In [7]:
ndf = pd.read_csv("https://trends.google.com/trends/explore?date=now%207-d&geo=US&q=%2Fm%2F02l5km,%2Fm%2F012gx2,%2Fm%2F0cqt90&hl=en#:~:text=help_outline-,file_download,-code")
#Look at google trends data

HTTPError: HTTP Error 429: unknown

## How did the report on Biden's memory change anything?